In [36]:
import pandas as pd
import boto3
import json
import os
from openai import OpenAI
from dotenv import load_dotenv
from tokencost import calculate_prompt_cost, calculate_completion_cost

bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-east-1")

ERROR:root:Failed to update token costs. Using static costs.
/Users/krystianpietrzak/Documents/ML/Phi3-FT/.venv/lib/python3.9/site-packages/tokencost/constants.py:61: RuntimeWarning: coroutine 'update_token_costs' was never awaited
  TOKEN_COSTS = TOKEN_COSTS_STATIC


In [11]:


model = "gpt-4-turbo"
prompt = [{ "role": "user", "content": '''You are a helpful assistant with access to this API:
            {
                "name": "book_flight",
                "description": "Book a flight from one location to another on particular date.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "start": {
                            "type": "string",
                            "description": "Where the user wants to travel from"
                        },
                        "destination": {
                            "type": "string",
                            "description": "Where the user wants to travel to."
                        }
                        "date": {
                            "type": "string",
                            "description": "Date of the flight"
                        }
                    },
                    "required": [
                        "start",
                        "destination",
                        "date"
                    ]
                }
            }
          Ask the user for all required parameters or return the API call.
          User: Hi my name is Krystian. I need to fly to Miami on 14.02.2024. Can you book a flight for me?'''
           }
           ]
completion = '''<function_call>
                book_flight
                {
                "start": "Paris",
                "destination": "Miami",
                "date": "12/04/2028"
                }
                </function_call>'''

prompt_cost = calculate_prompt_cost(prompt, model)
completion_cost = calculate_completion_cost(completion, model)

print(f"{prompt_cost} + {completion_cost} = {(prompt_cost + completion_cost)}")

0.00213 + 0.00129 = 0.00342


In [7]:
incomplete_user_query = '''SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -
{
    "name": "send_email",
    "description": "Send an email to a recipient",
    "parameters": {
        "type": "object",
        "properties": {
            "recipient": {
                "type": "string",
                "description": "The email address of the recipient"
            },
            "subject": {
...
        ]
    }
} 
USER: I need to send an email to my boss. The recipient is boss@company.com. The subject should be "Monthly Report" and the message should be "Please find attached the monthly report for your review." 
Simulate another user question that does not contain all the required details for the function call. Your response needs to start with USER: '''

In [ ]:
incomplete_user_query = f'{system}\n{user}\nSimulate another user question that does not contain all the required details for the function call. Your response needs to start with USER:'

In [2]:
df = pd.read_csv('train.csv')

In [46]:
def claude_inference(prompt):

    prompt_config = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature":0.1,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ],
            }
        ],
    }

    body = json.dumps(prompt_config)

    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_runtime.invoke_model(
        body=body, modelId=model_id, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    results = response_body.get("content")[0].get("text")

    price = (len(results)/4/1000)* 0.003+ (len(prompt)/4/1000)*0.015
    print(f'{price}$')
    
    return results

In [44]:
def gpt_inference(prompt):
    # Load API key from .env file
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    client = OpenAI(api_key=api_key)
    
    if api_key is None:
        raise ValueError("API key not found. Please set the OPENAI_API_KEY environment variable in the .env file.")
    
    completion = client.chat.completions.create(
                                            model="gpt-4o",
                                            temperature=0.1,
                                            max_tokens=4096,
                                            messages=[
                                                {"role": "user", "content": prompt}
                                            ]
                                            )
    
    response = completion.choices[0].message.content

    prompt_cost = calculate_prompt_cost(prompt, "gpt-4o")
    completion_cost = calculate_completion_cost(response, "gpt-4o")

    print(f"{(prompt_cost + completion_cost)}$")

    return response

In [47]:
claude_inference(incomplete_user_query)

0.00303675$


"USER: Can you help me send an email to my colleague about the project update? I want to make sure they're informed about our progress."

In [45]:
gpt_inference(incomplete_user_query)

0.001120$


'USER: I need to send an email to my colleague. The subject is "Meeting Update".'